# Maizeを利用してSBVSの前処理とドッキングシミュレーションを実行する



## 生成モデルから発生させた構造を確認する

SBVSを実施する前に構造を確認します。コードの詳細は割愛します。

In [ ]:
from rdkit import Chem
from rdkit.Chem import Draw
import pandas as pd

df = pd.read_csv('./TL_sampling.csv')
smiles_lst = [row['SMILES'] for idx, row in df.iterrows()]
# used first
smiles_lst = [s for s in smiles_lst][1:6]

mols = [Chem.MolFromSmiles(smi) for smi in smiles_lst]
Draw.MolsToGridImage(mols[:6], legends=[str(nll) for nll in df.NLL][:6], molsPerRow=5)

## Maizeのワークフローを構築する

XXで設定したtomlファイルからGypsumとAutoDock Vina用のノードを追加します。

In [ ]:
from pathlib import Path
from maize.core.workflow import Workflow
from maize.steps.io import LoadData, LogResult, Return
import os, sys

from maize.steps.mai.docking.adv import Vina
from maize.steps.mai.molecule import Gypsum
from maize.utilities.chem import IsomerCollection

In [ ]:
flow = Workflow(name='dock', level='info', cleanup_temp=True)
load = flow.add(LoadData[list[str]])
#void = flow.add(Void)
# 3D conf gen and protonate
embe = flow.add(Gypsum)
# run vina docking
dock = flow.add(Vina)
# receive docke results
retu = flow.add(Return[list[IsomerCollection]])

### AutoDock Vinaの設定

ドッキングシミュレーションの設定をします。ハンズオンでは説明しませんので詳細はAutoDock Vinaのドキュメントを参照してください。

In [ ]:
load.data.set(smiles_lst)
embe.n_variants.set(2)
embe.timeout.set(10)
# RIP3K <https://www.rcsb.org/structure/7MON>
dock.receptor.set(Path('/home/iwatobipen/dev/CBI/cheminfo_tutorial_20241028/data/2ito_fix_remh.pdbqt'))
dock.search_center.set((-56.643,  -6.844, -23.487))
dock.search_range.set((20.00, 20.00, 20.00))
# default vaule 8 seems too small ...
dock.exhaustiveness.set(16)

最後にノードを接続します。

In [ ]:
flow.connect(load.out, embe.inp)
flow.connect(embe.out, dock.inp)
flow.connect(dock.out,retu.inp)
flow.check()

In [ ]:
flow.visualize() # ワークフローを視覚化

In [ ]:
flow.execute() #実行

# 此処から先の説明はまだ

In [ ]:
!mkdir dock_res

In [ ]:
res=retu.get()
import numpy as np
from pathlib import Path
for midx, ms in enumerate(res):
    for cidx, m in enumerate(ms.molecules):
        m.to_sdf(Path(f'./dock_res/ok_{midx}_{cidx}.sdf'))

In [ ]:
res[0].molecules[1]